In [1]:
import sys
sys.path.append("..")
import msgspec
import faker
from tqdm.auto import tqdm
from typing import Any, cast
import pickle
import gc
import random


In [2]:
fake = faker.Faker("ru_RU")

**Ожидается, что заполнение узлов будет более полным при вставке упорядоченных данных**

In [3]:
from dkvs.bptree import BPTree
BPTree.HALF_INODE_SIZE = 31
BPTree.MAX_INODE_SIZE = BPTree.HALF_INODE_SIZE + 1
BPTree.HALF_LEAF_SIZE = 125
BPTree.MAX_LEAF_SIZE = BPTree.HALF_LEAF_SIZE + 1

In [4]:
class MyKey(msgspec.Struct, frozen=True, order=True):
    full_name: str

class Data(msgspec.Struct):
    data: Any

In [5]:
index = BPTree[MyKey, Data]()

In [6]:
faker.Faker.seed(1)
fake = faker.Faker("ru_RU")

In [7]:
data = []
gc.collect()

340

In [8]:
%%time
for i in tqdm(range(10_000_000)):
    # random_int = fake.random_int(0, 99_999_999)
    # val = f"{random_int:010} " + fake.name()
    val = f"{i:010} " + fake.name()
    result = index.insert(MyKey(val))
    if result[0] is None:
        print(f"Duplicate key: {val}: {result[1]}")
    data.append(val)

  0%|          | 0/10000000 [00:00<?, ?it/s]

CPU times: user 5min 39s, sys: 6.37 s, total: 5min 46s
Wall time: 5min 44s


**Создание дерева выполнилось практически в 2 раза быстрее**

In [9]:
index.validate(), gc.collect()

({}, 18)

In [10]:
index.find(MyKey('xxx'))

(None, None, None)

In [11]:
%%timeit
index.find(MyKey('0001677529 Шаров Леонтий Филатович'))

9.96 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [12]:
index.find(MyKey('0001677529 Шаров Леонтий Филатович'))

(None, None, None)

In [15]:
index.print_node(index.tree[0])


node-idx:0, prev:None, next:2, level: 0 
|-> MyKey(full_name='0000000000 Сазонов Арсений Дмитриевич')
|-> MyKey(full_name='0000000001 Мишин Радим Харитонович')
|-> MyKey(full_name='0000000002 Устинова Евфросиния Болеславовна')
|-> MyKey(full_name='0000000003 Никита Феликсович Кошелев')
|-> MyKey(full_name='0000000004 Агата Петровна Федотова')
|-> MyKey(full_name='0000000005 Данилова Феврония Борисовна')
|-> MyKey(full_name='0000000006 Алевтина Архиповна Козлова')
|-> MyKey(full_name='0000000007 Кузнецова Марина Даниловна')
|-> MyKey(full_name='0000000008 Алевтина Станиславовна Полякова')
|-> MyKey(full_name='0000000009 Полина Тимуровна Жукова')
|-> MyKey(full_name='0000000010 Ефим Фокич Селиверстов')
|-> MyKey(full_name='0000000011 Азарий Фёдорович Дьячков')
|-> MyKey(full_name='0000000012 Антонина Геннадиевна Евдокимова')
|-> MyKey(full_name='0000000013 тов. Ковалева Лора Сергеевна')
|-> MyKey(full_name='0000000014 Наталья Сергеевна Никонова')
|-> MyKey(full_name='0000000015 Евдокия 

In [16]:
index.min()[1]

MyKey(full_name='0000000000 Сазонов Арсений Дмитриевич')

In [17]:
index.max()[1]

MyKey(full_name='0009999999 Савина Анжелика Егоровна')

In [18]:
data[0:10]

['0000000000 Сазонов Арсений Дмитриевич',
 '0000000001 Мишин Радим Харитонович',
 '0000000002 Устинова Евфросиния Болеславовна',
 '0000000003 Никита Феликсович Кошелев',
 '0000000004 Агата Петровна Федотова',
 '0000000005 Данилова Феврония Борисовна',
 '0000000006 Алевтина Архиповна Козлова',
 '0000000007 Кузнецова Марина Даниловна',
 '0000000008 Алевтина Станиславовна Полякова',
 '0000000009 Полина Тимуровна Жукова']

In [19]:
random.shuffle(data)

In [20]:
%%time
for k in data:
    id = index.find(MyKey(k))[0]
    if id is not None:
        pass
        # print(f"found: {id} -> {index.keys[id]}")
    else:
        index.find(k)
        print(f"Key: {k} not found")

print("Done!")

Done!
CPU times: user 2min 27s, sys: 7.06 ms, total: 2min 27s
Wall time: 2min 27s


In [21]:
len(data) // (2 * 60 + 27)

68027